Learnings:

Given a custom type defined with `weave.type()` (& optionally a series of additional ops):
* You can run `weave.use` on the tail node and sucessfully get a response
* Running `weave.show` is broken with `TypeError: Object of type CUSTOM_TYPE is not JSON serializable`.
    * This is because `def show` needs to convert the requested object to a JSON document describing the object
        to the iframe (via URL parameter), which in turn needs to send the same (or modified) JSON back to a python
        engine. 
    * The first, not good solution, is to call `weave.save` on the object before passing it to the ops. This converts
        the object to a refrence, which can in fact be serialized. While this makes it work, it is not a reasonable
        ask of users.
    * This is becuase:
        a. when the ConstNode is serialized, it blindly assigns it's value to the value - which in these cases, are not primitive in themselves
        b. assuming we recurssively transform into a dictionary, we will reach some types that are not to_jsonable (like pytorch model)
        c. in those cases, we need to basically convert to a reference (similar to weave.save).
    * My problem is figuring out where to do this - in mappers, in the to_json of ConstNode? How does this relate to the instance_to/from_dict of the types? It feels like the later (which could be generallized in the ObjectType type). 
    * Followup, i want it to look like you are calling the constructor (like MyType(4, 5))
    
Potential Plan:
* Make @weave.type() constructors somehow a weave op that returns an instance of this type. This will preserve the UI of TypeName(params)
* Const nodes should attempt to save/serialize their value before writing it if it is of a special type.
    
```
from weave.show import _show_params

a = _show_params(addMagnitude(TypeExampleOne(45)))
a['weave_node'].to_json()

{'nodeType': 'output',
 'type': 'int',
 'fromOp': {'name': 'op-addMagnitude',
  'inputs': {'a': {'nodeType': 'const',
    'type': {'type': 'const',
     'valType': {'type': 'TypeExampleOne'},
     'val': TypeExampleOne(attr=45)},
    'val': TypeExampleOne(attr=45)}}}}
```

In [ ]:
# !rm -rf /tmp/local-artifacts

In [ ]:
import weave

In [ ]:
import random
## Avoids cache hits
def rand_int():
    return int(random.random() * 1e10)

def make_primitive():
    return rand_int()

In [ ]:
# weave.use - primitive
weave.use(make_primitive())

In [ ]:
# weave.show - primitive
weave.show(make_primitive())

In [ ]:
# weave.save - primitive
saved_primitive = weave.save(make_primitive())

In [ ]:
# weave.use - saved primitive
weave.use(saved_primitive)

In [ ]:
# weave.show - saved primitive
weave.show(saved_primitive)

In [ ]:
# weave.publish - primitive
# TODO: Does not work in CI
# published_primitive = weave.publish(make_primitive())

In [ ]:
# weave.use - published primitive
# TODO: Does not work in CI
# weave.use(published_primitive)

In [ ]:
# weave.show - published primitive
# TODO: Does not work in CI
# weave.show(published_primitive)

In [ ]:
# # Playground cell
# @weave.type()
# class FractionSimple:
#     numerator: int
#     denominator: int

# weave.show(FractionSimple(rand_int(), rand_int()))

In [ ]:
@weave.type()
class Fraction:
    numerator: int
    denominator: int

    @weave.op()
    def as_float(self) -> float:
        return self.numerator / self.denominator
    
    # UGG - this should be automatic!
    @weave.op()
    def get_numerator(self) -> int:
        return self.numerator
    
    # UGG - this should be automatic!
    @weave.op()
    def get_denominator(self) -> int:
        return self.denominator
    
@weave.op()
def show_fraction(frac: weave.Node[Fraction]) -> weave.panels.Group:
    return weave.panels.Group(
        items=[
            weave.panels.LabeledItem(label="numerator", item=frac.get_numerator()),
            weave.panels.LabeledItem(label="denominator", item=frac.get_denominator()),
            weave.panels.LabeledItem(label="fraction", item=frac.as_float()),
        ],
    ) 

def make_fraction():
    return Fraction(rand_int(), rand_int())

In [ ]:
# weave.use - primitive
weave.use(make_fraction())

In [ ]:
# weave.show - primitive
# TODO: This requires a kernal restart
weave.show(make_fraction())

In [ ]:
# weave.save - primitive
saved_fraction = weave.save(make_fraction())

In [ ]:
# weave.use - saved primitive
weave.use(saved_fraction)

In [ ]:
# weave.show - saved primitive
weave.show(saved_fraction)

In [ ]:
# weave.publish - primitive
# TODO: Does not work in CI
# published_fraction = weave.publish(make_fraction())

In [ ]:
# weave.use - published primitive
# TODO: Does not work in CI
# weave.use(published_fraction)

In [ ]:
# weave.show - published primitive
# TODO: Does not work in CI
# weave.show(published_fraction)

In [ ]:
custom_result = make_fraction().as_float()
custom_result